In [1]:
import pandas as pd

city_df = pd.read_csv("https://raw.githubusercontent.com/datadesk/california-coronavirus-data/master/latimes-place-totals.csv")
city_df.x = city_df.x.abs() * -1  # Correcting a mistake in longitude
city_df.x = city_df.x.fillna(0)
city_df.y = city_df.y.fillna(0)
city_df

,date,county,fips,place,confirmed_cases,note,x,y
0,2020-03-16,Los Angeles,37,Alhambra,2,NaN,-118.135504,34.083961
1,2020-03-16,Los Angeles,37,Arcadia,1,NaN,-118.037297,34.134186
2,2020-03-16,Los Angeles,37,Beverly Hills,1,NaN,-118.402109,34.078543
3,2020-03-16,Los Angeles,37,Boyle Heights,5,NaN,-118.205330,34.038150
4,2020-03-16,Los Angeles,37,Carson,1,NaN,-118.255878,33.837391
...,...,...,...,...,...,...,...,...
28195,2020-05-10,Tulare,107,Woodlake,11,NaN,-119.096931,36.413681
28196,2020-05-10,Yolo,113,Davis,20,NaN,-121.738056,38.553889
28197,2020-05-10,Yolo,113,West Sacramento,58,NaN,-121.530278,38.580556
28198,2020-05-10,Yolo,113,Winters and unincorporated,11,NaN,-121.970833,38.525000


New python package for finding zipcode called uszipcode
Can find zipcode based on coordinates
https://pypi.org/project/uszipcode/

In [2]:
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True)
result = search.by_coordinates(38.678611, -121.773333, radius=30, returns=1)
result[0]

SimpleZipcode(zipcode='95776', zipcode_type='Standard', major_city='Woodland', post_office_city='Woodland, CA', common_city_list=['Woodland'], county='Yolo County', state='CA', lat=38.7, lng=-121.7, timezone='Pacific', radius_in_miles=11.0, area_code_list=['530'], population=21902, population_density=237.0, land_area_in_sqmi=92.43, water_area_in_sqmi=1.12, housing_units=6824, occupied_housing_units=6464, median_home_value=248000, median_household_income=61599, bounds_west=-121.781382, bounds_east=-121.629202, bounds_north=38.803028, bounds_south=38.553296)

Select unique cities and look for zip code in a new table

In [3]:
city_data_df = pd.DataFrame()
city_data_df = pd.concat([city_df.county, city_df.place, city_df.x, city_df.y], axis=1)
city_data_df = city_data_df.drop_duplicates()

def search_zip(df):
    # If place name contains zip code, returns the zip code directly
    if df.place[0:5].isnumeric():
        return df.place[0:5]
    if df.y == 0:
        return 0
    result = search.by_coordinates(df.y, df.x, radius=30, returns=1)
    return result[0].zipcode

city_data_df['zip'] = city_data_df.apply(search_zip, axis=1)
city_data_df = city_data_df.reset_index(drop=True)
city_data_df['city_id'] = city_data_df.index
city_data_df

,county,place,x,y,zip,city_id
0,Los Angeles,Alhambra,-118.135504,34.083961,91803,0
1,Los Angeles,Arcadia,-118.037297,34.134186,91006,1
2,Los Angeles,Beverly Hills,-118.402109,34.078543,90212,2
3,Los Angeles,Boyle Heights,-118.205330,34.038150,90033,3
4,Los Angeles,Carson,-118.255878,33.837391,90745,4
...,...,...,...,...,...,...
940,San Mateo,Woodside,-122.253900,37.429900,94061,940
941,Alameda,Eden Area MAC (Ashland,0.000000,0.000000,0,941
942,Kern,93501: Mojave,-118.194488,35.023791,93501,942
943,Los Angeles,Hidden Hills,-118.661203,34.163749,91302,943


In [4]:
city_df = pd.merge(city_df, city_data_df, "inner", on=['county', 'place', 'x', 'y'])
city_df

,date,county,fips,place,confirmed_cases,note,x,y,zip,city_id
0,2020-03-16,Los Angeles,37,Alhambra,2,NaN,-118.135504,34.083961,91803,0
1,2020-03-17,Los Angeles,37,Alhambra,1,NaN,-118.135504,34.083961,91803,0
2,2020-03-18,Los Angeles,37,Alhambra,1,NaN,-118.135504,34.083961,91803,0
3,2020-03-19,Los Angeles,37,Alhambra,2,NaN,-118.135504,34.083961,91803,0
4,2020-03-20,Los Angeles,37,Alhambra,3,NaN,-118.135504,34.083961,91803,0
...,...,...,...,...,...,...,...,...,...,...
28195,2020-05-10,Kern,29,93501: Mojave,1,NaN,-118.194488,35.023791,93501,942
28196,2020-05-09,Los Angeles,37,Hidden Hills,1,NaN,-118.661203,34.163749,91302,943
28197,2020-05-10,Los Angeles,37,Hidden Hills,1,NaN,-118.661203,34.163749,91302,943
28198,2020-05-09,San Luis Obispo,79,San Luis Obispo (city),17,NaN,0.000000,0.000000,0,944


In [6]:
import numpy as np

# Suppress warnings
pd.options.mode.chained_assignment = None

city_id = city_df.city_id.unique()

for i in city_id:
    temp_df = city_df[city_df.city_id == i]
    temp_df['daily_cases'] = temp_df['confirmed_cases'].diff().fillna(0)
    temp_df['rate'] = temp_df['confirmed_cases'].pct_change().fillna(0)
    temp_df['rate7day'] = temp_df['rate'].rolling(window=7).mean().fillna(0)
    temp_df = temp_df.replace(np.inf, 0)
    if i == 0:
        new_df = temp_df
    else:
        new_df = pd.concat([new_df, temp_df])
new_df

,date,county,fips,place,confirmed_cases,note,x,y,zip,city_id,daily_cases,rate,rate7day
0,2020-03-16,Los Angeles,37,Alhambra,2,NaN,-118.135504,34.083961,91803,0,0.0,0.0,0.0
1,2020-03-17,Los Angeles,37,Alhambra,1,NaN,-118.135504,34.083961,91803,0,-1.0,-0.5,0.0
2,2020-03-18,Los Angeles,37,Alhambra,1,NaN,-118.135504,34.083961,91803,0,0.0,0.0,0.0
3,2020-03-19,Los Angeles,37,Alhambra,2,NaN,-118.135504,34.083961,91803,0,1.0,1.0,0.0
4,2020-03-20,Los Angeles,37,Alhambra,3,NaN,-118.135504,34.083961,91803,0,1.0,0.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28195,2020-05-10,Kern,29,93501: Mojave,1,NaN,-118.194488,35.023791,93501,942,0.0,0.0,0.0
28196,2020-05-09,Los Angeles,37,Hidden Hills,1,NaN,-118.661203,34.163749,91302,943,0.0,0.0,0.0
28197,2020-05-10,Los Angeles,37,Hidden Hills,1,NaN,-118.661203,34.163749,91302,943,0.0,0.0,0.0
28198,2020-05-09,San Luis Obispo,79,San Luis Obispo (city),17,NaN,0.000000,0.000000,0,944,0.0,0.0,0.0


In [7]:
new_df.to_csv("city_data.csv", index=False)
